## Configuring Gradient Boosting

### XGBoost's hyperparameters
XGBoost has a multitude of tuning parameters that are split into three categories:
- General Parameters, which guide the overall functioning
- Booster Parameters, which guide the indiviual boosters at each step
- Learning Task Parameters, which guide the opitmization performed
Not all are important to our project, but we will need to spend some time looking at the tuning parameters to figure out the best ones for out project.

There are too many for us to list all, butt some of the most common ones, and most likely that we will use, are:
- nthread
- seed
- silent
- subsample
- max_delta_step
- missing
- scale_pos_weight
- booster
- gbtree
- max_depth
- n_estimators
- colsample_bytree
- reg_lambda
- colsample_bylevel
- objective
- leaning_rate
- base_score
- min_child_weight
- reg_alpha
- gamma

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('/users/kdeuser/Desktop/CMPU366/FinalProject/mbti_1.csv') 

In [11]:
## set all MBTI letters to correspond to 0 and 1s
mbtipersonalities = {'I':0, 'E':1, 'N':0, 'S':1, 'T':0, 'F':1, 'J':0, 'P':1}
## format of MBTI perdonality types
mbtip_list = [{0:'I', 1:'E'}, {0:'N', 1:'E'}, {0:'T', 1:'F'}, {0:'J', 1:'P'}]

## make mbti personalities to binary vectors
def mbti_to_bin_vec(mbti_pers):
    return [mbtipersonalities[i] for i in mbti_pers]

## change back from binary vectors to the mbti personalities
def bin_vec_to_mbti(mbti_pers):
    ## needs to be a string
    s = ""
    for i, j in enumerate(mbti_pers):
        s += mbtip_list[i][j]
    return s

check = data.head(10)
mbti_binvec  = np.array([mbti_to_bin_vec(i) for i in check.type])
print("Binarized MBTI list:\n%s"% mbti_binvec)

binvec_mbti = np.array([bin_vec_to_mbti(i) for i in mbti_binvec])
print("Back to MBTI:  \n%s" % binvec_mbti)

##check with og data.head
print("Original MBTI: \n%s" % data.head(10).type)

Binarized MBTI list:
[[0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 0 0]
 [1 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 1]]
Back to MBTI:  
['INFJ' 'ENTP' 'INTP' 'INTJ' 'ENTJ' 'INTJ' 'INFJ' 'INTJ' 'INFJ' 'INTP']
Original MBTI: 
0    INFJ
1    ENTP
2    INTP
3    INTJ
4    ENTJ
5    INTJ
6    INFJ
7    INTJ
8    INFJ
9    INTP
Name: type, dtype: object


In [12]:
# to preprocss the data we have to make everything lowercase, remove
# nonalphabetic words, urls and overly common words (e.g. a, the, at, etc.)
# and lemmatice the words

### ADD REMOVEING STOPPWORDS and mbti types
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import re

## list of mbti types
mbti_types_list = ['ESTP', 'ESFP', 'ENFP', 'ENTP', 'ESTJ', 'ESFJ', 'ENFJ', 'ENTJ',
                  'ISTJ', 'ISFJ', 'INFJ', 'INTJ', 'ISTP', 'ISFP', 'INFP', 'INTP']
mbti_types_list = [i.lower() for i in mbti_types_list]

##choose stopwords
stopWOrds = stopwords.words("english")

lemmatizer = WordNetLemmatizer()

def preprocessing(data, remove_mbti=True, remove_stopwords=True):
    personalities = []
    posts = []
    datalength = len(data)
    i = 0
    
    ## iterate through the rows 
    for row in data.iterrows():
        i += 1
        if (i % 500 == 0 or i ==1 or i == datalength):
            print("%s of %s rows" % (i, datalength))
        
        ## go through current row
        comments = row[1].posts
        ## remove urls, after % is ascii charcters so 2 bits
        removeurls = re.sub('(http|https|ftp)://(?:[A-Za-z]|[0-9]|[$-_&@+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', comments)
        ## remove non words
        removenonwords = re.sub('[^A-Za-z]', ' ', removeurls)
        removeextraspace = re.sub('  +',' ', removenonwords)
        ## make lowercase 
        changed = removeextraspace.lower()
        
        #lemmatize basedoff of includiong stopwords or not 
        if remove_stopwords:
            changed = " ".join([lemmatizer.lemmatize(w) for w in changed.split(' ') if w not in stopWOrds])
        else:
            ## lemmatize
            changed = " ".join([lemmatizer.lemmatize(w) for w in changed.split(' ')])
        
        ## remove mbti types
        if remove_mbti:
            for m in mbti_types_list:
                changed = changed.replace(m,"")
                
        ## make mbti typesbinary vectors
        bin_vecs = mbti_to_bin_vec(row[1].type)
        personalities.append(bin_vecs)
        posts.append(changed)
        
    posts = np.array(posts)
    personalities = np.array(personalities)
    return posts, personalities
        

In [13]:
posts, personalities = preprocessing(data)

1 of 8675 rows
500 of 8675 rows
1000 of 8675 rows
1500 of 8675 rows
2000 of 8675 rows
2500 of 8675 rows
3000 of 8675 rows
3500 of 8675 rows
4000 of 8675 rows
4500 of 8675 rows
5000 of 8675 rows
5500 of 8675 rows
6000 of 8675 rows
6500 of 8675 rows
7000 of 8675 rows
7500 of 8675 rows
8000 of 8675 rows
8500 of 8675 rows
8675 of 8675 rows


In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE



count_vec = CountVectorizer(stop_words="english",
                            analyzer = "word",
                            ngram_range=(1,1),
                            max_df=0.9,
                            min_df=0.1,
                            max_features=None)


count_train = count_vec.fit(posts)
print("CountVectorizer...")
# should create and return a count-vectorized output of docs
posts_count = count_vec.fit_transform(posts)


tfidf_izer = TfidfTransformer()

print("tf-idf...")
posts_tfidf =  tfidf_izer.fit_transform(posts_count).toarray()                           
                             
#vectorizer = TfidfVectorizer()
#vectors = vectorizer.fit_transform([documentA, documentB])
#feature_names = vectorizer.get_feature_names()
#dense = vectors.todense()
#denselist = dense.tolist()
#df = pd.DataFrame(denselist, columns=feature_names)

CountVectorizer...
tf-idf...


In [15]:
# the MBTI type indicators
mbti_type_ind = ["IE: Introversion (I) - Extroversion (E)",
                "NS: Intuition (N) - Sensing (S)",
                "FT: Feeling(F) - Thinking (T)",
                "JP: Judging (J) - Percieving (P)"]

for m in range(len(mbti_type_ind)):
    print(mbti_type_ind[m])
    
mbti1row = bin_vec_to_mbti(personalities[0,:])

IE: Introversion (I) - Extroversion (E)
NS: Intuition (N) - Sensing (S)
FT: Feeling(F) - Thinking (T)
JP: Judging (J) - Percieving (P)


In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')


X = posts_tfidf

#we need to train each of the mbti type indicators
for m in range(len(mbti_type_ind)):
    print("%s ..." % mbti_type_ind[m])
    
    #for each type indicator we train a different Y
    Y = personalities[:,m]
    
    # next we split the data into train and test sets
    #unsure how to choose the random seed or test size
    seed = 50
    test_size = 0.50
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    #fit model on the training data
    model = XGBClassifier()
    model.fit(X_train, Y_train)
        
    #make predictions for the test data
    Y_prediction = model.predict(X_test)
    predictions = [round(value) for value in Y_prediction]
    
    #check the accuracy
    accuracy = accuracy_score(Y_test, predictions)
    print(" - %s Accuracy: %.2f%%" % (mbti_type_ind[m], accuracy * 100.0))

IE: Introversion (I) - Extroversion (E) ...
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ...
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ...
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ...
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%


In [17]:
#some of the xgbparams we may look at 
default_get_xgb_params = model.get_xgb_params()
print(default_get_xgb_params)

{'objective': 'binary:logistic', 'max_depth': 3, 'learning_rate': 0.1, 'subsample': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'scale_pos_weight': 1, 'reg_alpha': 0, 'n_estimators': 100, 'silent': 1, 'seed': 0, 'missing': None, 'reg_lambda': 1, 'base_score': 0.5, 'nthread': 1, 'gamma': 0, 'colsample_bytree': 1, 'max_delta_step': 0, 'min_child_weight': 1}


We will start iuth using the parameters:
- n_estimators: number of trees to build
- max_depth: the maximum. depth of a tree, 
    - the larger the value the more complex the model is and more likely to overfit 
- nthread: the number of parallel threads running XGBoost
- learning_rate (also known as eta): step size shrinkage, in order to prevent overfitting. 

In [18]:
#set up some of the parameters for xgboost
param={}

#good ones to choose initially would be n_estimators, max_depth, n_thread, learning_rate
param['n_estimators']=100
#max_depth default is 6 
param['max_depth']=2
#number of parallel threads running XGBoost
param['nthread']=8
#learning rate default is 0.3
param['learning_rate']=0.2

#training the MBTI type indicators individually
for m in range(len(mbti_type_ind)):
    print("%s ... " % (mbti_type_ind[m]))
    
    Y= personalities[:,m]
    
    #split into train and test sets
    seed = 50
    test_size = 0.50
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    #fit model on the training data
    model = XGBClassifier(**param)
    model.fit(X_train, Y_train)
        
    #make predictions for the test data
    Y_prediction = model.predict(X_test)
    predictions = [round(value) for value in Y_prediction]
    
    #check the accuracy
    accuracy = accuracy_score(Y_test, predictions)
    print(" - %s Accuracy: %.2f%%" % (mbti_type_ind[m], accuracy * 100.0))
    
    

IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.78%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.68%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.44%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 64.62%


First attempts of parameter tuning did little with the accuracy not improving at all, with the rough estimates of accuracy still being:
- IE: 77%
- NS: 86%
- FT: 73%
- JP: 65%
note: played around with the parameter, including setting n_estimates to 50 instead of 100, which reduced our accuracy by around 1% for each mbti type.

Can also try:
- subsample: ratio of the training istances, reducing means that less is sampled of the training data before growing trees
- colsample_bytree: percentage of features used per tree. The higher it is the more likely it will lead to overfitting
- num_parallel_tree: number of parallel trees constructed during each iteration


#### Playing around with parameters
We are going through testing different parameters to see which result in the best improvement of accuracy.
The accuracy before parameters were added, for test_set=0.5 (one run):
- IE: 77.41%
- NS: 86.58%
- TF: 73.28%
- JP: 65.35%

In [20]:
param={}


param_list=['n_estimators', 'max_depth', 'nthread', 'learning_rate', 'subsample',
          'colsample_bytree', 'num_parallel_tree', 'num_feature', 'seed',
          'silent', 'max_delta_step', 'scale_pos_weight',
          'reg_lambda', 'colsample_bylevel', 'objective', 
          'base_score', 'min_child_weight', 'reg_alpha', 'gamma']

#previous value was 100
param['n_estimators']=90 #no real change when set to 100

#max_depth default is 6 , was set at 3
param['max_depth']=4  #not a big improvement

#number of parallel threads running XGBoost, was set at 1
param['nthread']=2 #no real change

#learning rate default is 0.3, was set at 0.1
param['learning_rate']=0.2 #slight changes

#default is 1
param['subsample']=0.75 #a bit of change

#default is 1
param['colsample_bytree']= 0.75 #slight changes

#default is 1
param['num_parallel_tree'] = 2 #no real change

#set automatically by XGBoost
param['num_feature']= 25 #no real change

#default 0
param['seed']= 21

#belongs to verbosity?
param['silent']= 2

#default 0
param['max_delta_step']= 2

#set at None
#param['missing']=

#set at 1
param['scale_pos_weight']= 2

#default is gbtree
#param['booster'] = "dgblinear" #no real changes with gblinear or dart

#default is 1
param['reg_lambda']= 2

#default is 1
param['colsample_bylevel']=0.5

#set at "binary:logistoc"
param['objective']= "reg:logistic"

#set at 0.5
param['base_score']= 0.7

#default 1
param['min_child_weight']= 2

#set at 0
param['reg_alpha']= 1

#default is 0
param['gamma']= 1


#go throguh all params
for p in range(len(param_list)):
    params={}  
    params[param_list[p]] = param[param_list[p]]
    print(param_list[p])
    #training the MBTI type indicators individually
    for m in range(len(mbti_type_ind)):
        print("%s ... " % (mbti_type_ind[m]))

        Y= personalities[:,m]

        #split into train and test sets
        seed = 50
        test_size = 0.50

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

        #fit model on the training data
        model = XGBClassifier(**params)
        model.fit(X_train, Y_train)

        #make predictions for the test data
        Y_prediction = model.predict(X_test)
        predictions = [round(value) for value in Y_prediction]

        #check the accuracy
        accuracy = accuracy_score(Y_test, predictions)
        print(" - %s Accuracy: %.2f%%" % (mbti_type_ind[m], accuracy * 100.0))



n_estimators
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.57%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.08%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.31%
max_depth
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.57%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.49%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.72%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.21%
nthread
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 


n_estimators
 - IE: 77.57%
 - NS: 86.63%
 - FT: 73.08%
 - JP: 65.31%

max_depth
 - IE: 77.57%
 - NS: 86.49%
 - FT: 73.72%
 - JP: 65.21%

nthread
 - IE: 77.41%
 - NS: 86.58%
 - FT: 73.28%
 - JP: 65.35%

learning_rate
 - IE: 77.32%
 - NS: 86.33%
 - FT: 73.91%
 - JP: 65.95%

subsample
 - IE: 77.55%
 - NS: 86.61%
 - FT: 73.72%
 - JP: 65.86%

colsample_bytree
 - IE: 77.69%
 - NS: 86.63%
 - FT: 73.70%
 - JP: 65.44%

num_parallel_tree
 - IE: 77.41%
 - NS: 86.58%
 - FT: 73.28%
 - JP: 65.35%

num_feature
 - IE: 77.41%
 - NS: 86.58%
 - FT: 73.28%
 - JP: 65.35%

seed
 - IE: 77.41%
 - NS: 86.58%
 - FT: 73.28%
 - JP: 65.35%

silent
 - IE: 77.41%
 - NS: 86.58%
 - FT: 73.28%
 - JP: 65.35%

max_delta_step
 - IE: 77.41%
 - NS: 86.58%
 - FT: 73.28%
 - JP: 65.35%

scale_pos_weight
 - IE: 75.86%
 - NS: 86.26%
 - FT: 68.42%
 - JP: 63.12%

reg_lambda 
 - IE: 77.80%
 - NS: 86.56%
 - FT: 73.65%
 - JP: 65.05%

colsample_bylevel
 - IE: 77.57%
 - NS: 86.63%
 - FT: 73.35%
 - JP: 65.31%

objective
 - IE: 77.41%
 - 86.58%
 - 73.28%
 - 65.35%

base_score
 - IE: 77.22% 
 - NS: 86.63%
 - FT: 73.17%
 - JP: 65.56%

min_child_weight
 - IE: 77.59%
 - NS: 86.61%
 - FT: 73.86%
 - JP: 64.78%

reg_alpha
 - IE: 77.69%
 - NS: 86.61%
 - FT: 73.72% 
 - JP: 65.15%

gamma
 - IE: 77.69%
 - NS: 86.61%
 - FT: 73.37%
 - JP: 65.63%

n_estimators = 90
 - IE: 77.57% 
 - NS: 86.63%
 - FT: 73.08% 
 - JP: 65.31%
 
max_depth = 4
 - IE: 77.50%
 - NS: 86.51%
 - FT: 73.58%
 - JP: 65.31%
 
nthread = 2
 - IE: 77.50%
 - NS: 86.51%
 - FT: 73.58%
 - JP: 65.31%
 
learning_rate = 0.2
 - IE: 76.88%
 - NS: 86.17% 
 - FT: 73.03%
 - JP: 64.08%
 
subsample = 0.75
 - IE: 76.23%
 - NS: 86.10%
 - FT: 74.00%
 - JP: 64.25%
 
colsample_bytree = 0.75
 - IE: 77.13%
 - NS: 86.10%
 - FT: 73.74%
 - JP: 62.93%
 
num_parallel_tree = 2
 - IE: 77.48%
 - NS: 86.40%
 - FT: 74.69%
 - JP: 64.75%
 
num_feature = 25
 - IE: 77.48%
 - NS: 86.40%
 - FT: 74.69%
 - JP: 64.75%
 
seed = 21
 - IE: 77.06%
 - NS: 86.24%
 - FT: 75.20%
 - JP: 63.95%
 
silent = 2
 - IE: 77.06%
 - NS: 86.24%
 - FT: 75.20%
 - JP: 63.95%
 
max_delta_step = 2
 - IE: 77.06%
 - NS: 86.24%
 - FT: 75.20%
 - JP: 63.95%
 
scale_pos_weight = 2
 - IE: 75.70% 
 - NS: 85.82%
 - FT: 72.01%
 - JP: 65.28%
 
reg_lambda = 2
 - IE: 75.68% 
 - NS: 85.82%
 - FT: 73.03%
 - JP: 63.99%
 
colsample_bylevel = 0.5
 - IE: 76.56%
 - NS: 85.34%
 - FT: 72.61%
 - JP: 64.04%
 
objective = "reg:logistic"
 - IE: 76.56%
 - NS: 85.34%
 - FT: 72.61%
 - JP: 64.04%
 
base_score = 0.7
 - IE: 76.33%
 - NS: 85.43%
 - FT: 72.57%
 - JP: 65.05%
 
min_child_weight = 2
 - IE: 76.03%
 - NS: 85.34%
 - FT: 72.25%
 - JP: 63.53%
 
reg_alpha = 1
 - IE: 75.96%
 - NS: 85.36%
 - FT: 72.22%
 - JP: 64.59%
 
gamma = 1
 - IE: 76.35%
 - NS: 85.52%
 - FT: 72.43% 
 - JP: 63.95%

In [56]:
param={}

param_list = ['num_feature', 
              'seed', 'silent', 'max_delta_step', 'scale_pos_weight', 
              'reg_lambda', 'min_child_weight', 'reg_alpha', 'gamma']
random_num_list=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]



#max_depth default is 6 , was set at 3
param['max_depth']=4  #not a big improvement

#number of parallel threads running XGBoost, was set at 1
param['nthread']=2 #no real change

#default is 1
param['num_parallel_tree'] = 2 #no real change

#set automatically by XGBoost
param['num_feature']= 25 #no real change

#default 0
param['seed']= 21

#belongs to verbosity?
param['silent']= 2

#default 0
param['max_delta_step']= 2


#set at 1
param['scale_pos_weight']= 2

#default is 1
param['reg_lambda']= 2

#default 1
param['min_child_weight']= 2

#set at 0
param['reg_alpha']= 1

#default is 0
param['gamma']= 1

#go throguh all params

for p in range(len(param_list)):
    for r in range(len(random_num_list)):
        params={}  
        params[param_list[p]] = random_num_list[r]
        print('%s = %s' % (param_list[p], random_num_list[r]))
        #training the MBTI type indicators individually
        for m in range(len(mbti_type_ind)):
            print("%s ... " % (mbti_type_ind[m]))

            Y= personalities[:,m]

            #split into train and test sets
            seed = 50
            test_size = 0.50

            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

            #fit model on the training data
            model = XGBClassifier(**params)
            model.fit(X_train, Y_train)

            #make predictions for the test data
            Y_prediction = model.predict(X_test)
            predictions = [round(value) for value in Y_prediction]

            #check the accuracy
            accuracy = accuracy_score(Y_test, predictions)
            print(" - %s Accuracy: %.2f%%" % (mbti_type_ind[m], accuracy * 100.0))



num_feature = 0
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%
num_feature = 1
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%
num_feature = 2
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - T

 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%
silent = 0
IE: Introversion (I) - Extroversion (E) ... 
[19:30:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:42] src/tree

[19:30:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:30:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_

[19:30:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:30:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[19:30:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_

[19:30:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_

[19:30:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ... 
[19:30:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:30:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:31:00] src/t

[19:31:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:31:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:31:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_

 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%
max_delta_step = 4
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%
max_delta_step = 5
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accur

 - NS: Intuition (N) - Sensing (S) Accuracy: 86.56%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.65%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.05%
reg_lambda = 3
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.66%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.24%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.74%
reg_lambda = 4
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.62%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.67%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.51%
re

 - FT: Feeling(F) - Thinking (T) Accuracy: 73.72%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.15%
reg_alpha = 2
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.48%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 72.98%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%
reg_alpha = 3
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.52%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.97%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.42%
reg_alpha = 4
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extrov

### num_feature

num_feature = 0
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%


### seed

seed = 0
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%


### silent

silent = 0
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

silent = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

silent = 2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

silent = 3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

silent = 4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

silent = 5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

silent = 6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

silent = 7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

silent = 8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

silent = 9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

silent = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

### max_delta_step

max_delta_step = 0
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

max_delta_step = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

max_delta_step = 2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

max_delta_step = 3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

max_delta_step = 4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

max_delta_step = 5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

max_delta_step = 6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

max_delta_step = 7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

max_delta_step = 8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

max_delta_step = 9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

max_delta_step = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

### scale_pos_weight

scale_pos_weight = 0
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 46.59%
 - JP: Judging (J) - Percieving (P) Accuracy: 39.65%

scale_pos_weight = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

scale_pos_weight = 2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 75.86%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.26%
 - FT: Feeling(F) - Thinking (T) Accuracy: 68.42%
 - JP: Judging (J) - Percieving (P) Accuracy: 63.12%

scale_pos_weight = 3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 71.35%
 - NS: Intuition (N) - Sensing (S) Accuracy: 84.39%
 - FT: Feeling(F) - Thinking (T) Accuracy: 64.59%
 - JP: Judging (J) - Percieving (P) Accuracy: 61.78%

scale_pos_weight = 4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 65.35%
 - NS: Intuition (N) - Sensing (S) Accuracy: 81.65%
 - FT: Feeling(F) - Thinking (T) Accuracy: 62.03%
 - JP: Judging (J) - Percieving (P) Accuracy: 61.27%

scale_pos_weight = 5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 58.92%
 - NS: Intuition (N) - Sensing (S) Accuracy: 78.28%
 - FT: Feeling(F) - Thinking (T) Accuracy: 61.02%
 - JP: Judging (J) - Percieving (P) Accuracy: 61.02%

scale_pos_weight = 6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 54.08%
 - NS: Intuition (N) - Sensing (S) Accuracy: 74.83%
 - FT: Feeling(F) - Thinking (T) Accuracy: 59.61%
 - JP: Judging (J) - Percieving (P) Accuracy: 61.04%

scale_pos_weight = 7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 48.76%
 - NS: Intuition (N) - Sensing (S) Accuracy: 71.30%
 - FT: Feeling(F) - Thinking (T) Accuracy: 59.15%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.88%

scale_pos_weight = 8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 45.78%
 - NS: Intuition (N) - Sensing (S) Accuracy: 67.96%
 - FT: Feeling(F) - Thinking (T) Accuracy: 57.98%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.79%

scale_pos_weight = 9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 43.68%
 - NS: Intuition (N) - Sensing (S) Accuracy: 65.12%
 - FT: Feeling(F) - Thinking (T) Accuracy: 57.61%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.81%

scale_pos_weight = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 41.68%
 - NS: Intuition (N) - Sensing (S) Accuracy: 62.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 57.12%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.81%

### reg_lambda

reg_lambda = 0
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.64%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 72.66%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.68%

reg_lambda = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

reg_lambda = 2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.80%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.56%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.65%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.05%

reg_lambda = 3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.66%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.24%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.74%

reg_lambda = 4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.62%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.67%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.51%

reg_lambda = 5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.57%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.44%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.72%

reg_lambda = 6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.78%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.97%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.68%

reg_lambda = 7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.64%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.07%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.61%

reg_lambda = 8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.62%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.54%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.79%

reg_lambda = 9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.43%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.54%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.49%

reg_lambda = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.66%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.42%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.72%


### min_child_weight

min_child_weight = 0
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.49%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.94%

min_child_weight = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

min_child_weight = 2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.59%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.86%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.78%

min_child_weight = 3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.29%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.33%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.89%

min_child_weight = 4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.49%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.56%

min_child_weight = 5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.59%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.35%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.17%

min_child_weight = 6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.39%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.70%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.68%

min_child_weight = 7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.52%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.86%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.85%

min_child_weight = 8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.48%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.68%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.07%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.05%

min_child_weight = 9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.32%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.47%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.38%

min_child_weight = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.52%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.21%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.51%

### reg_alpha

reg_alpha = 0
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

reg_alpha = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.69%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.72%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.15%

reg_alpha = 2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.48%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 72.98%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

reg_alpha = 3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.52%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.97%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.42%

reg_alpha = 4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.52%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.61%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.98%

reg_alpha = 5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.36%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.88%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.31%

reg_alpha = 6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.02%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.63%
 - JP: Judging (J) - Percieving (P) Accuracy: 66.04%

reg_alpha = 7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.22%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.47%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.72%

reg_alpha = 8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.55%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.33%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.68%

reg_alpha = 9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.18%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.56%

reg_alpha = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.36%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.44%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.84%

### gamma 
gamma = 0
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

gamma = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.69%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.37%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.63%

gamma = 2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.71%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.37%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.08%

gamma = 3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.01%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.05%

gamma = 4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.12%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.91%

gamma = 5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.59%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.65%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.98%

gamma = 6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.66%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.91%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.40%

gamma = 7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.35%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.10%

gamma = 8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.52%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.49%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.71%

gamma = 9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.78%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.81%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.91%

gamma = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.62%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.79%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.96%

In [54]:
param={}

param_list = ['learning_rate', 'subsample', 'colsample_bytree', 
              'colsample_bylevel', 'base_score']
random_num_list=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]


#learning rate default is 0.3, was set at 0.1
param['learning_rate']=0.2 #slight changes

#default is 1
param['subsample']=0.75 #a bit of change

#default is 1
param['colsample_bytree']= 0.75 #slight changes

#default is 1
param['colsample_bylevel']=0.5

#set at 0.5
param['base_score']= 0.7




#go throguh all params
for p in range(len(param_list)):
    for r in range(len(random_num_list)):
        params={}  
        params[param_list[p]] = random_num_list[r]
        print('%s = %s' % (param_list[p], random_num_list[r]))
        #training the MBTI type indicators individually
        for m in range(len(mbti_type_ind)):
            print("%s ... " % (mbti_type_ind[m]))

            Y= personalities[:,m]

            #split into train and test sets
            seed = 50
            test_size = 0.50

            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

            #fit model on the training data
            model = XGBClassifier(**params)
            model.fit(X_train, Y_train)

            #make predictions for the test data
            Y_prediction = model.predict(X_test)
            predictions = [round(value) for value in Y_prediction]

            #check the accuracy
            accuracy = accuracy_score(Y_test, predictions)
            print(" - %s Accuracy: %.2f%%" % (mbti_type_ind[m], accuracy * 100.0))



learning_rate = 0.1
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%
learning_rate = 0.2
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.32%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.33%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.91%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.95%
learning_rate = 0.3
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.20%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 85.82%
FT: Fe

 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.59%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.47%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.12%
colsample_bytree = 0.3
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.58%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.10%
colsample_bytree = 0.4
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.62%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (

 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.43%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.05%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.17%
base_score = 0.4
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.62%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.54%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.05%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.03%
base_score = 0.5
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy:

XGBoostError: b'[19:12:56] src/objective/./regression_loss.h:62: Check failed: base_score > 0.0f && base_score < 1.0f base_score must be in (0,1) for logistic loss\n\nStack trace returned 8 entries:\n[bt] (0) 0   libxgboost.dylib                    0x000000201a35e3bd dmlc::StackTrace() + 301\n[bt] (1) 1   libxgboost.dylib                    0x000000201a35e15f dmlc::LogMessageFatal::~LogMessageFatal() + 47\n[bt] (2) 2   libxgboost.dylib                    0x000000201a35dc19 dmlc::LogMessageFatal::~LogMessageFatal() + 9\n[bt] (3) 3   libxgboost.dylib                    0x000000201a3db0e9 xgboost::obj::LogisticRegression::ProbToMargin(float) + 201\n[bt] (4) 4   libxgboost.dylib                    0x000000201a370817 xgboost::LearnerImpl::LazyInitModel() + 887\n[bt] (5) 5   libxgboost.dylib                    0x000000201a37c60b XGBoosterUpdateOneIter + 91\n[bt] (6) 6   libffi.6.dylib                      0x0000000104e22884 ffi_call_unix64 + 76\n[bt] (7) 7   ???                              

### learning_rate
learning_rate = 0.1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

learning_rate = 0.2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.32%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.33%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.91%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.95%

learning_rate = 0.3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.20%
 - NS: Intuition (N) - Sensing (S) Accuracy: 85.82%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.14%
 - JP: Judging (J) - Percieving (P) Accuracy: 63.42%

learning_rate = 0.4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 76.37%
 - NS: Intuition (N) - Sensing (S) Accuracy: 85.27%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.61%
 - JP: Judging (J) - Percieving (P) Accuracy: 63.21%

learning_rate = 0.5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 75.63%
 - NS: Intuition (N) - Sensing (S) Accuracy: 84.44%
 - FT: Feeling(F) - Thinking (T) Accuracy: 72.50%
 - JP: Judging (J) - Percieving (P) Accuracy: 62.03%

learning_rate = 0.6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 74.25%
 - NS: Intuition (N) - Sensing (S) Accuracy: 84.97%
 - FT: Feeling(F) - Thinking (T) Accuracy: 71.12%
 - JP: Judging (J) - Percieving (P) Accuracy: 62.13%

learning_rate = 0.7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 73.81%
 - NS: Intuition (N) - Sensing (S) Accuracy: 84.07%
 - FT: Feeling(F) - Thinking (T) Accuracy: 71.12%
 - JP: Judging (J) - Percieving (P) Accuracy: 61.66%

learning_rate = 0.8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 72.75%
 - NS: Intuition (N) - Sensing (S) Accuracy: 83.82%
 - FT: Feeling(F) - Thinking (T) Accuracy: 71.23%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.33%

learning_rate = 0.9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 72.54%
 - NS: Intuition (N) - Sensing (S) Accuracy: 83.47%
 - FT: Feeling(F) - Thinking (T) Accuracy: 69.64%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.00%

learning_rate = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 72.87%
 - NS: Intuition (N) - Sensing (S) Accuracy: 83.06%
 - FT: Feeling(F) - Thinking (T) Accuracy: 69.82%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.63%

### subsample

subsample = 0.1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 75.93%
 - NS: Intuition (N) - Sensing (S) Accuracy: 85.52%
 - FT: Feeling(F) - Thinking (T) Accuracy: 71.76%
 - JP: Judging (J) - Percieving (P) Accuracy: 61.60%

subsample = 0.2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 76.37%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.42%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.08%
 - JP: Judging (J) - Percieving (P) Accuracy: 63.65%

subsample = 0.3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.38%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.86%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.87%

subsample = 0.4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.51%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.02%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.80%

subsample = 0.5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.69%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.35%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.09%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.54%

subsample = 0.6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.73%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.16%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.63%

subsample = 0.7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.32%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.56%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.42%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.82%

subsample = 0.8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.77%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.87%

subsample = 0.9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.66%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.14%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.94%

subsample = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%



### colsample_by_tree

colsample_bytree = 0.1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.72%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.32%

colsample_bytree = 0.2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.59%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.47%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.12%

colsample_bytree = 0.3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.58%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.10%

colsample_bytree = 0.4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.62%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.77%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.10%

colsample_bytree = 0.5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.99%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.35%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.08%

colsample_bytree = 0.6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.73%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.72%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.89%

colsample_bytree = 0.7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.39%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.74%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.40%

colsample_bytree = 0.8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.85%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.09%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.12%

colsample_bytree = 0.9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.22%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.72%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.73%

colsample_bytree = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%



### colsample_bylevel

colsample_bylevel = 0.1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.59%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.65%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.28%

colsample_bylevel = 0.2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.47%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.68%

colsample_bylevel = 0.3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.64%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.02%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.03%

colsample_bylevel = 0.4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.71%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.65%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.12%

colsample_bylevel = 0.5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.57%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.35%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.31%

colsample_bylevel = 0.6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.64%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.61%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.89%

colsample_bylevel = 0.7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.62%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.08%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.40%

colsample_bylevel = 0.8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.19%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.01%

colsample_bylevel = 0.9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.39%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.08%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.17%

colsample_bylevel = 1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%


### base_score

base_score = 0.1
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.80%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 72.50%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.49%

base_score = 0.2
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.29%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.63%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.71%

base_score = 0.3
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.43%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.05%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.17%

base_score = 0.4
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.62%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.54%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.05%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.03%

base_score = 0.5
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

base_score = 0.6
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.46%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 72.68%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.24%

base_score = 0.7
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.22%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.17%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.56%

base_score = 0.8
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.46%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.49%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.36%

base_score = 0.9
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.64%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.61%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.79%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.80%


In [53]:
param = {}
param_list=['n_estimators', 'num_feature', 'seed', 'gamma']

random_num_list=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]

#previous value was 100
param['n_estimators']=90 #no real change when set to 100

#set automatically by XGBoost
param['num_feature']= 25 #no real change

#default 0
param['seed']= 21


#default is 0
param['gamma']= 1


#go throguh all params
for p in range(len(param_list)):
    for r in range(len(random_num_list)):
        params={}  
        params[param_list[p]] = random_num_list[r]
        print('%s = %s' % (param_list[p], random_num_list[r]))
        #training the MBTI type indicators individually
        for m in range(len(mbti_type_ind)):
            print("%s ... " % (mbti_type_ind[m]))

            Y= personalities[:,m]

            #split into train and test sets
            seed = 50
            test_size = 0.50

            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

            #fit model on the training data
            model = XGBClassifier(**params)
            model.fit(X_train, Y_train)

            #make predictions for the test data
            Y_prediction = model.predict(X_test)
            predictions = [round(value) for value in Y_prediction]

            #check the accuracy
            accuracy = accuracy_score(Y_test, predictions)
            print(" - %s Accuracy: %.2f%%" % (mbti_type_ind[m], accuracy * 100.0))


n_estimators = 10
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.22%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 66.62%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 64.41%
n_estimators = 20
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.09%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 69.13%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 64.38%
n_estimators = 30
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.13%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(

 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%
seed = 10
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%
seed = 20
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Ju

 - JP: Judging (J) - Percieving (P) Accuracy: 60.35%
gamma = 110
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 61.60%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 60.35%


### n_estimators
n_estimators = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.22%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.65%
 - FT: Feeling(F) - Thinking (T) Accuracy: 66.62%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.41%

n_estimators = 20
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.09%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 69.13%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.38%

n_estimators = 30
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.13%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 70.52%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.64%

n_estimators = 40
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.18%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 71.21%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.96%

n_estimators = 50
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.36%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 71.92%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.78%

n_estimators = 60
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.48%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 72.13%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.19%

n_estimators = 70
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 72.80%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.65%

n_estimators = 80
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.32%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.03%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.21%

n_estimators = 90
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.57%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.08%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.31%

n_estimators = 100
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

n_estimators = 110
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.39%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.77%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.03%


### num_feature

num_feature = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 20
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 30
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 40
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 50
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 60
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 70
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 80
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 90
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 100
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

num_feature = 110
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

### seed

seed = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 20
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 30
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 40
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 50
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 60
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 70
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 80
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 90
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 100
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

seed = 110
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%


### gamma

gamma = 10
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.62%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.79%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.96%

gamma = 20
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.09%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 71.23%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.43%

gamma = 30
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 69.96%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.27%

gamma = 40
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 68.37%
 - JP: Judging (J) - Percieving (P) Accuracy: 61.27%

gamma = 50
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 66.74%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.35%

gamma = 60
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 66.25%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.35%

gamma = 70
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 65.12%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.35%

gamma = 80
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 64.98%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.35%

gamma = 90
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 64.59%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.35%

gamma = 100
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 62.93%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.35%

gamma = 110
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 61.60%
 - JP: Judging (J) - Percieving (P) Accuracy: 60.35%

### Booster

In [21]:
param={}

param_list = ['booster']
random_num_list=['gbtree', 'gblinear', 'dart']


#default is gbtree
param['booster'] = "dgblinear" #no real changes with gblinear or dart


#go throguh all params
params={}  
for p in range(len(param_list)):
    for r in range(len(random_num_list)):
        params={}  
        params[param_list[p]] = random_num_list[r]
        print('%s = %s' % (param_list[p], random_num_list[r]))
        #training the MBTI type indicators individually
        for m in range(len(mbti_type_ind)):
            print("%s ... " % (mbti_type_ind[m]))

            Y= personalities[:,m]

            #split into train and test sets
            seed = 50
            test_size = 0.50

            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

            #fit model on the training data
            model = XGBClassifier(**params)
            model.fit(X_train, Y_train)

            #make predictions for the test data
            Y_prediction = model.predict(X_test)
            predictions = [round(value) for value in Y_prediction]

            #check the accuracy
            accuracy = accuracy_score(Y_test, predictions)
            print(" - %s Accuracy: %.2f%%" % (mbti_type_ind[m], accuracy * 100.0))



booster = gbtree
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%
booster = gblinear
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 53.41%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 60.35%
booster = dart
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(F) 

booster = gbtree
 - IE: 77.41%
 - NS: 86.58%
 - FT: 73.28%
 - JP: 65.35%

booster = gblinear
 - IE: 77.06%
 - NS: 86.63%
 - FT: 53.41%
 - JP: 60.35%

booster = dart
 - IE: 77.41%
 - NS: 86.58%
 - FT: 73.28%
 - JP: 65.35%

### Objective

In [52]:
param={}

param_list = ['objective']
#random_num_list=['reg:squarederror', 'reg:squaredlogerror', 'reg:logistic',
#                'reg:pseudohubererror', 'binary:logistic', 'binary:logitraw',
#                'binary:hinge', 'count:poisson', 'survival:cox', 'survival:aft',
#                'aft_loss_distribution', 'multi:softmax', 'multi:softprob',
#                'rank:pairwise', 'rank:ndcg', 'rank:map', 'rank:gamma', ' reg:tweedie']

#set at "binary:logistoc"
param['objective']= "reg:gamma"



#go throguh all params
#training the MBTI type indicators individually
for m in range(len(mbti_type_ind)):
    print("%s ... " % (mbti_type_ind[m]))

    Y= personalities[:,m]

    #split into train and test sets
    seed = 50
    test_size = 0.50

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    #fit model on the training data
    model = XGBClassifier(**param)
    model.fit(X_train, Y_train)

    #make predictions for the test data
    Y_prediction = model.predict(X_test)
    predictions = [round(value) for value in Y_prediction]

    #check the accuracy
    accuracy = accuracy_score(Y_test, predictions)
    print(" - %s Accuracy: %.2f%%" % (mbti_type_ind[m], accuracy * 100.0))



IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 76.83%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 85.89%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 65.28%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 60.05%


objective = reg:logistic
 - IE: 77.41%
 - NS: 86.58%
 - FT: 73.28%
 - JP: 65.35%

objective = binary:logistic
 - IE: 77.41%
 - NS: 86.58%
 - FT: 73.28%
 - JP: 65.35%

objective = binary:logitraw
 - IE: 77.13%
 - NS: 86.63%
 - FT: 72.25%
 - JP: 61.27%
 
objective = binary:hinge
 - IE: 75.89%
 - NS: 86.40%
 - FT: 71.16%
 - JP: 64.50%

objective = count:poisson
 - IE: 77.27%
 - NS: 86.68%
 - FT: 72.84%
 - JP: 64.85%

objective = survival:cox
 - IE: 22.94%
 - NS: 13.37%
 - FT: 53.41%
 - JP: 60.35%

objective = rank:pairwise
 - IE: 69.78%
 - NS: 72.36%
 - FT: 72.68%
 - JP: 64.22%

objective = rank:ndcg
 - IE: 77.06%
 - NS: 86.63%
 - FT: 46.59%
 - JP: 39.65%

objective = rank:map 
 - IE: 77.06%
 - NS: 86.63%
 - FT: 46.59%
 - JP: 39.65%

objective = rank:gamma
 - IE: 76.83%
 - NS: 85.89%
 - FT: 65.28%
 - JP: 60.05%

objective = reg:tweedie
 - IE: 77.13%
 - NS: 86.63%
 - FT: 70.79%
 - JP: 63.65%

### n_estimators

In [55]:
param={}

param_list = ['n_estimators']
random_num_list=[0, 50, 100, 150, 200, 250, 300]


#default is gbtree
param['n_estimators']= 100 #no real changes with gblinear or dart


#go throguh all params
params={}  
for p in range(len(param_list)):
    for r in range(len(random_num_list)):
        params={}  
        params[param_list[p]] = random_num_list[r]
        print('%s = %s' % (param_list[p], random_num_list[r]))
        #training the MBTI type indicators individually
        for m in range(len(mbti_type_ind)):
            print("%s ... " % (mbti_type_ind[m]))

            Y= personalities[:,m]

            #split into train and test sets
            seed = 50
            test_size = 0.50

            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

            #fit model on the training data
            model = XGBClassifier(**params)
            model.fit(X_train, Y_train)

            #make predictions for the test data
            Y_prediction = model.predict(X_test)
            predictions = [round(value) for value in Y_prediction]

            #check the accuracy
            accuracy = accuracy_score(Y_test, predictions)
            print(" - %s Accuracy: %.2f%%" % (mbti_type_ind[m], accuracy * 100.0))



n_estimators = 0
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 46.59%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 39.65%
n_estimators = 50
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.36%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
FT: Feeling(F) - Thinking (T) ... 
 - FT: Feeling(F) - Thinking (T) Accuracy: 71.92%
JP: Judging (J) - Percieving (P) ... 
 - JP: Judging (J) - Percieving (P) Accuracy: 64.78%
n_estimators = 100
IE: Introversion (I) - Extroversion (E) ... 
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
NS: Intuition (N) - Sensing (S) ... 
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
FT: Feeling(

n_estimators = 0
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.06%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 46.59%
 - JP: Judging (J) - Percieving (P) Accuracy: 39.65%

n_estimators = 50
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.36%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.63%
 - FT: Feeling(F) - Thinking (T) Accuracy: 71.92%
 - JP: Judging (J) - Percieving (P) Accuracy: 64.78%

n_estimators = 100
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.41%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.58%
 - FT: Feeling(F) - Thinking (T) Accuracy: 73.28%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.35%

n_estimators = 150
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.64%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.54%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.69%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.17%

n_estimators = 200
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.29%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.42%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.39%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.44%

n_estimators = 250
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.50%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.42%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.55%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.38%

n_estimators = 300
 - IE: Introversion (I) - Extroversion (E) Accuracy: 77.29%
 - NS: Intuition (N) - Sensing (S) Accuracy: 86.45%
 - FT: Feeling(F) - Thinking (T) Accuracy: 74.46%
 - JP: Judging (J) - Percieving (P) Accuracy: 65.10%